In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Nlp-codealong').getOrCreate()

In [7]:
data = spark.read.csv('smsspamcollection/SMSSpamCollection',inferSchema=True,sep='\t')


In [8]:
data.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [9]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [10]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [11]:
from pyspark.sql.functions import length

In [13]:
data = data.withColumn('length',length(data['text']))

In [14]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [15]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [16]:
from pyspark.ml.feature import CountVectorizer,Tokenizer,StopWordsRemover,IDF,StringIndexer

In [18]:
tokenizer = Tokenizer(inputCol='text',outputCol='token_text')

In [19]:
stop_remove= StopWordsRemover(inputCol='token_text',outputCol='stop_token')

In [20]:
count_vec = CountVectorizer(inputCol='stop_token',outputCol='c_vec')

In [21]:
idf=IDF(inputCol='c_vec',outputCol='tf_idf')

In [22]:
ham_spam_to_numeric = StringIndexer(inputCol='class',outputCol='label')

In [23]:
from pyspark.ml.feature import VectorAssembler

In [24]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

In [25]:
from pyspark.ml.classification import NaiveBayes

In [26]:
nb = NaiveBayes()

In [29]:
from pyspark.ml import Pipeline

In [30]:
data_prep_pipe  = Pipeline(stages=[ham_spam_to_numeric,tokenizer,
                                  stop_remove,count_vec,idf,clean_up])

In [33]:
clean_data = data_prep_pipe.fit(data).transform(data)

In [34]:
clean_data.columns

Exception ignored in: <object repr() failed>
Traceback (most recent call last):
  File "/Users/apple/anaconda/lib/python3.6/site-packages/pyspark/ml/wrapper.py", line 105, in __del__
    SparkContext._active_spark_context._gateway.detach(self._java_obj)
AttributeError: 'Tokenizer' object has no attribute '_java_obj'


['class',
 'text',
 'length',
 'label',
 'token_text',
 'stop_token',
 'c_vec',
 'tf_idf',
 'features']

In [35]:
clean_data = clean_data.select('label','features')

In [36]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [37]:
train,test= clean_data.randomSplit([0.7,0.3])

In [39]:
spam_detector = nb.fit(train)

In [40]:
test_results= spam_detector.transform(test)

In [41]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,41,...|[-1056.6567715924...|[1.0,1.4762647515...|       0.0|
|  0.0|(13424,[0,1,3,9,1...|[-570.59632303258...|[0.99999999999993...|       0.0|
|  0.0|(13424,[0,1,9,14,...|[-538.66405586966...|[1.0,5.5212865522...|       0.0|
|  0.0|(13424,[0,1,14,78...|[-692.47193296639...|[1.0,3.3990908001...|       0.0|
|  0.0|(13424,[0,1,18,20...|[-841.20489164160...|[1.0,1.4055077613...|       0.0|
|  0.0|(13424,[0,1,20,27...|[-980.97829747287...|[0.99999999999858...|       0.0|
|  0.0|(13424,[0,1,21,27...|[-1005.2574617301...|[1.0,1.2011884027...|       0.0|
|  0.0|(13424,[0,1,30,12...|[-614.29863795156...|[1.0,6.4235662458...|       0.0|
|  0.0|(13424,[0,1,46,17...|[-1144.7324167822...|[8.22567265049587...|       1.0|
|  0.0|(13424,[0

In [42]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [43]:
acc_eval = MulticlassClassificationEvaluator()

In [44]:
acc = acc_eval.evaluate(test_results)

In [45]:
print(acc)

0.9178489829671928
